## Run Transformer models on hand-labeled policy field classification tweets

In [32]:
import pandas as pd
import os
import re
from dotenv import load_dotenv
from openai import OpenAI

In [33]:
load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY")) # get API key from .env file

In [34]:
# set a seed for sampling
SEED = 1349

In [35]:
# read data
data = pd.read_csv("CAP_sample_coded.tsv", 
                   sep="\t", 
                     dtype={"_id": str} # _id column as string to avoid scientific notation
                     )
data
                   

,_id,verkehr,entwicklung,gesellschaft,innere_sicherheit,verteidigung,aeusseres,digitalisierung_technik,soziales,umwelt,...,kultur_medien_sport,gesundheit,landwirtschaft_ernaehrung,bildung_forschung,wirtschaft,none,intercoder_sample,coder,wirtschaft_finanzen_merge,_source.text
0,922866835887984641,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,"Der ""schwarze Block"" hat übrigens heute (links..."
1,923412288308236288,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,VW-Chef Müller fordert Superministerium Wirtsc...
2,923452548442460160,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,300.000 Polizisten sorgen Tag für Tag für unse...
3,926501188236869634,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,"~ 1️⃣2️⃣,5️⃣ Mio. Deutsche haben per Zweitstim..."
4,927635670352441346,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,"#AfD-Doppelmandatsträger:\r\nSpangenberg, Mros..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2394,1522118401937793033,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,True,0,True,Eil: Gericht erklärt #Wirecard-Bilanzen für ni...
2395,1567473763280719872,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,True,0,True,"Fraktionsvize @mueller_sepp stellt fest, für O..."
2396,1569284933209710594,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,0,False,Ehren Flyeralarm :) https://t.co/5BPNl1VSeu
2397,1633203373263527940,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,True,0,False,RT @HarjitSajjan: Very productive meeting with...


In [36]:
# we remove the categories "wirtschaft" (economy) and "haushalt_finanzen" (finance) in favor of the compound category "wirtschaft_finanzen"
data = data.drop(columns=["wirtschaft", "haushalt_finanzen"])
# rename "wirtschaft_finanzen_merge" to "wirtschaft_finanzen"
data = data.rename(columns={"wirtschaft_finanzen_merge": "wirtschaft_finanzen"})
data

,_id,verkehr,entwicklung,gesellschaft,innere_sicherheit,verteidigung,aeusseres,digitalisierung_technik,soziales,umwelt,...,arbeit,kultur_medien_sport,gesundheit,landwirtschaft_ernaehrung,bildung_forschung,none,intercoder_sample,coder,wirtschaft_finanzen,_source.text
0,922866835887984641,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,"Der ""schwarze Block"" hat übrigens heute (links..."
1,923412288308236288,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,VW-Chef Müller fordert Superministerium Wirtsc...
2,923452548442460160,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,300.000 Polizisten sorgen Tag für Tag für unse...
3,926501188236869634,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,"~ 1️⃣2️⃣,5️⃣ Mio. Deutsche haben per Zweitstim..."
4,927635670352441346,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,"#AfD-Doppelmandatsträger:\r\nSpangenberg, Mros..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2394,1522118401937793033,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,0,True,Eil: Gericht erklärt #Wirecard-Bilanzen für ni...
2395,1567473763280719872,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,0,True,"Fraktionsvize @mueller_sepp stellt fest, für O..."
2396,1569284933209710594,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,0,False,Ehren Flyeralarm :) https://t.co/5BPNl1VSeu
2397,1633203373263527940,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,True,0,False,RT @HarjitSajjan: Very productive meeting with...


In [37]:
# pivot data to long format
data_long = data.melt(id_vars=["_id", "intercoder_sample", "coder", "_source.text"], 
                      var_name="label")

data_long = data_long[data_long.value == True] # only keep tweets labeled as a certain category (drop False values)

# remove value and coder column
data_long = data_long.drop(columns=["value", "coder"])

data_long



,_id,intercoder_sample,_source.text,label
802,1623756907532910593,False,Das #Deutschlandticket ist ein richtiger Schri...,verkehr
806,1400799340021616640,False,Das Jobticket ist nun seit einem Monat am Star...,verkehr
825,1044258236663304193,False,Der Antrag zu einem Sozialticket für Bus und B...,verkehr
853,1483489042691940357,False,Die Stadt Leverkusen inkl. Oberbürgermeister u...,verkehr
866,1109875138516959234,False,Diese Kampagne bringt wohl mehr als die üblich...,verkehr
...,...,...,...,...
40770,1360263802294579201,True,Der #Unternehmerlohn für Kulturschaffende und ...,wirtschaft_finanzen
40774,1457645827078434818,True,"Schätzungen gehen davon aus, dass jedes Jahr m...",wirtschaft_finanzen
40778,1522118401937793033,True,Eil: Gericht erklärt #Wirecard-Bilanzen für ni...,wirtschaft_finanzen
40779,1567473763280719872,True,"Fraktionsvize @mueller_sepp stellt fest, für O...",wirtschaft_finanzen


In [38]:
# nr of unique documents that are labeled. Note: not all document in the data had been labeled, hence the reduced number!
data_long._id.nunique()

597

In [39]:
# get duplicated rows by _id (i.e. documents fitting multiple labels)
multi_label = data_long[data_long.duplicated('_id', keep = False)]
multi_label._id.nunique() # 181 out of 597 documents have more than one label

181

In [51]:
 # use multilabel data for testing (keep only ID and text)
sample = multi_label.drop_duplicates(subset="_id", keep="first")[['_id', '_source.text']]
# add the labels from data_long to the sample, after combining multiple labels into one string
sample = sample.merge(multi_label.groupby("_id")["label"].apply(lambda x: " ".join(x)).reset_index(), on="_id")
sample

,_id,_source.text,label
0,1400799340021616640,Das Jobticket ist nun seit einem Monat am Star...,verkehr umwelt arbeit
1,1044258236663304193,Der Antrag zu einem Sozialticket für Bus und B...,verkehr soziales
2,1295351591890018306,Es geht um #KfZZulassung. https://t.co/DClf0npcfu,verkehr innere_sicherheit
3,1148476943991484416,"Fahrpreiserhöhungen sind nie sexy, aber das #T...",verkehr wirtschaft_finanzen
4,956834625028022274,Gesunder Menschenverstand hat sich durchgesetz...,verkehr kultur_medien_sport
...,...,...,...
176,1371897354635329540,Gute Insights heute @GRUENE_LSA mit @Connylue ...,gesundheit bildung_forschung
177,1253224498171650050,RT @WolfgangWiehle: Linksgrüne Abbruchfantasie...,gesundheit wirtschaft_finanzen
178,1572559296621350914,Hier ist jetzt auch ganz offiziell die Nicht-A...,bildung_forschung wirtschaft_finanzen
179,1032945820222582787,Dampfturbinen werden zur Stromgewinnung klassi...,bildung_forschung wirtschaft_finanzen


In [120]:
# take a random sample of 10 documents as test input
test_input = sample.sample(10, random_state=SEED)

In [42]:
# dictionaries for labels in English and German
labels_en = {
'verkehr':'Transportation', 
'entwicklung':'Foreign Aid', 
'gesellschaft':'Civil Rights, Minority Issues and Civil Liberties', 
'innere_sicherheit':'Domestic Policy',
'verteidigung':'Defence Policy', 
'aeusseres':'Foreign Policy', 
'digitalisierung_technik':'Digitalisation, Science and Technology', 
'soziales':'Social Welfare',
'umwelt':'Environmental Policy', 
'europa':'European Policy', 
'arbeit':'Labour, Employment and Immigration', 
'kultur_medien_sport':'Culture, Media and Sports', 
'gesundheit':'Healthcare',
'landwirtschaft_ernaehrung':'Agriculture', 
'bildung_forschung':'Education and Research', 
'wirtschaft_finanzen':'Economy, Energy and Finance',
'none':'none',
}

labels_de = {
'verkehr':'Transport und Verkehr', 
'entwicklung':'Entwicklungszusammenarbeit', 
'gesellschaft':'Bürger- und Freiheitsrechte, Minderheitsrechte', 
'innere_sicherheit':'Innenpolitik und Sicherheit',
'verteidigung':'Verteidigungspolitik', 
'aeusseres':'Außenpolitik', 
'digitalisierung_technik':'Digitalisierung, Wissenschaft und Technologie', 
'soziales':'Sozialpolitik',
'umwelt':'Umweltpolitik', 
'europa':'Europapolitik', 
'arbeit':'Arbeitwesen und Arbeitsmigration', 
'kultur_medien_sport':'Kultur, Medien und Sport', 
'gesundheit':'Gesundheitswesen',
'landwirtschaft_ernaehrung':'Landwirtschaft', 
'bildung_forschung':'Bildung und Forschung', 
'wirtschaft_finanzen':'Volkswirtschaft, Energie und Bankwesen',
'none':'keine Kategorie',
}

In [43]:
# definitions
definitions_en = { # these are AI translated labels (from definitions_de)
    'verkehr': 'Public transport and safety; Road construction, traffic, and safety; Airports, airlines, air traffic, and air traffic control; Rail transport and safety; Shipbuilding and maritime transport; Public works, infrastructure development; Research and development', 
    'entwicklung': 'Development and foreign aid; Developing countries', 
    'gesellschaft': 'Ethnic minorities and discrimination; Discrimination based on gender and sexual orientation; Age discrimination; Discrimination based on disability or illness; Voting rights and electoral matters; Freedom of speech, religion, and assembly; Privacy rights and freedom of information; Anti-government behavior and activities; Church law; Immigration, refugees, expellees, and naturalization; Relations with the GDR and occupying powers as well as reunification; Economic issues, trust and currency issues; Stasi matters and other civil and human rights; Social policy (including standard of living adjustment)', 
    'innere_sicherheit': 'Nationwide crime-fighting agencies; Economic crime and organized crime; Combating and controlling illegal drugs; Court administration; Prisons; Juvenile crime and justice; Child abuse and child protection; Police, fire protection, and gun control; Civil and criminal law; Riots and crime prevention; Disaster relief; Domestic terrorism; Federal, state, and municipal relations; Efficiency and bureaucratic oversight; Postal service; Social benefits for public employees; Nomination, appointment, and dismissal in the public service; Currency and coinage, federal printing office; Public procurement and fraud; Management and sale of public buildings and land; Tax administration; Resignation and political scandals; Parliamentary and ministerial relations; Regulation of political activity, elections, and campaigns; Government ethics; Census and federal statistics; Berlin/Bonn; Lawsuits against the government; Holidays',
    'verteidigung': 'Defense policy and alliances; Federal intelligence service and espionage; Military readiness; Arms control and non-proliferation of nuclear weapons; Military support and arms supply to other states; Armed forces and their families, military court; War returnees and veterans; Military procurement; Military facilities, construction, land acquisition; Conscription and reservists; Military-related environmental protection; Civil defense; Civilian employees of the armed forces and defense industry, closure of military bases; Oversight of military contracts and suppliers; Deployment and war matters; Lawsuits against the armed forces; Research and development', 
    'aeusseres': 'Trade negotiations, disputes, and agreements; Export promotion and regulation, trade bank; International private sector investments; Productivity and competitiveness, balance of payments; Import duties and restrictions; Exchange rates; International environmental issues and resource management; International financial and economic development; Soviet Union, former republics, and Eastern Europe; Africa; Western Europe; Latin America; International channels; North America; Asia, Pacific region, Australia, and Japan; Middle East; Human rights; International organizations; Terrorism and kidnapping; Diplomats, embassies, German citizens abroad, foreign diplomats in Germany, passports', 
    'digitalisierung_technik': 'DARA, ESA, and space research; Satellites and commercial use of space; Technology transfer, international scientific cooperation; Telecommunications and infrastructure; Weather forecasting, German weather service, oceanography; Computer industry, computer security, information security; Research and development', 
    'soziales': 'Family matters; Food aid; Support for low-income households; Elderly care; Support for disabled and incapacitated individuals; Social services and volunteer organizations; Child benefits, parental allowance, and other support; Urban housing; Urban development; Rural housing; Rural economic development; Housing assistance for low- and middle-income citizens; Housing for seniors and disabled individuals; Homelessness',
    'umwelt': 'Drinking water safety; Waste disposal; Regulation, treatment, and disposal of hazardous waste and toxins; Air pollution, global warming, and noise pollution; Recycling; Environmental risks and pollutants in buildings; Species and forest protection; Pollution and protection of navigable waterways and coasts; Landscape and water protection; Research and development; Parks, monuments, historical sites, recreation, and natural treasures; Public land and forest management; Research and management of public waters', 
    'europa': 'European Union and common market', 
    'arbeit': 'Work safety and protection; Job training; Personal benefits for employees; Trade unions and works councils; Working conditions and policies; Child and youth labor, initiatives against youth unemployment; Maternity protection, parental leave, and childcare; Foreign workers', 
    'kultur_medien_sport': 'Art and culture promotion; Sports regulation and betting supervision; Broadcasting, television, and cable providers', 
    'gesundheit': 'Healthcare reform; Health insurance reform, benefits, and costs; Regulation of the pharmaceutical industry, medical technology, and clinical laboratories; Regulation and funding of medical facilities; Funding and regulation of insurance and doctors; Medical liability, fraud, and abuse; Medical workforce and training; Prevention, communicable diseases, and health promotion; Infants and children; Mental health, developmental delays; Long-term care, home care, rehabilitation, care for the incurable; Coverage and costs of prescription drugs; Other or multiple benefits and measures; Tobacco abuse, treatment, and education; Alcohol abuse and treatment; Drug and medication abuse, education, and treatment; Research and development',
    'landwirtschaft_ernaehrung': 'Agricultural trade; State subsidies for farmers and ranchers, agricultural disaster insurance; Food control and safety; Agricultural marketing and promotion; Animal and plant diseases, and pest control; Animal welfare, handling of animals; Environmental issues caused by agriculture; Fisheries; Agricultural research and development', 
    'bildung_forschung': 'Higher education; Primary and secondary education; Equal educational opportunities; Vocational training; Special education and support for learning disabilities; Education and excellence promotion; Research and development', 
    'wirtschaft_finanzen': 'Inflation, prices, and interest rates; Unemployment rate, unemployment; Central bank; State budget and national debt; Taxes, tax policy, and tax reforms; Structural and industrial policy; Price control and stabilization; Nuclear energy and nuclear power plants; Electricity and hydropower; Natural gas and oil; Coal; Alternative and renewable energy; Energy conservation and savings; Research and development; Banks and financial supervision; Securities and stock market supervision; Consumer finance, mortgages, and credit cards; Insurance supervision; Bankruptcy; Merger and antitrust supervision, corporate governance; Small and medium-sized enterprises; Copyrights and patents; Tourism; Consumer protection and consumer fraud',
    'none': 'None of the categories are suitable',
}

definitions_de = {
'verkehr':'Öffentlicher Verkehr und Sicherheit; Straßenbau, -verkehr und -sicherheit; Flughäfen, Fluglinien, Luftverkehr, und Flugsicherung; Schienenverkehr und Sicherheit; Schiffbau und Seeverkehr; Öffentliche Bauarbeiten, Infrastrukturentwicklung; Forschung und Entwicklung', 
'entwicklung':'Entwicklungs- und Auslandshilfe; Entwicklungsländer', 
'gesellschaft':'Ethnische Minderheiten und Diskriminierung; Diskriminierung aufgrund von Geschlecht und sexueller Ausrichtung; Altersdiskriminierung; Diskriminierung aufgrund von Behinderung oder Krankheit; Wahlrecht und Wahlrechtsangelegenheiten; Rede-, Glaubens- und Versammlungsfreiheit; Persönlichkeitsrechte und Informationsfreiheit; regierungsfeindliche/s Verhalten und Tätigkeit; Kirchenrecht; Einwanderung, Flüchtlinge, Aussiedler und Einbürgerung; Beziehung zur DDR und Besatzungsmächten sowie Einigung; Wirtschaftliche Fragen, Treuhand- und Währungsfragen; Stasi-Angelegenheiten und andere Bürger und Freiheitsrechte; Sozialpolitik (inkl. Anpassung des Lebensstandards)', 
'innere_sicherheit':'Bundesweite Behörden für Verbrechensbekämpfung; Wirtschaftskriminalität und organisiertes Verbrechen; Bekämpfung und Kontrolle von illegalen Drogen; Gerichtsverwaltung; Gefängnisse; Jugendkriminalität und justizwesen; Kindesmissbrauch und Kinderschutz; Polizei, Brandschutz und Waffenkontrolle; Zivil- und Strafrecht; Ausschreitungen und Verbrechensvorbeugung; Katastrophenhilfe; Innerdeutscher Terrorismus; Bund-, Länder-, Gemeindebeziehungen; Rentabilität und bürokratische Aufsicht; Postwesen; Soziale Leistungen für Angestellte des öffentlichen Dienstes; Nominierung, Ernennung und Entlassung im öffentlichen Dienst; Währungs- und Münzwesen, Bundesdruckerei; Öffentliches Auftragswesen und Betrug; Verwaltung und Verkauf von öffentlichen Gebäuden und Grundstücken; Steuerverwaltung; Amtsniederlegung und Politische Skandale; Parliamentarische und ministeriale Beziehungen; Regelung politischer Betätigung, Wahlen und Wahlkämpfe; Regierungs-Ethik; Volkszählung und Bundesstatistiken; Berlin/Bonn; Klagen gegen die Regierung; Feiertage',
'verteidigung':'Verteidigungspolitik und Verteidigungsbündnisse; Bundesnachrichtendienst und Spionage; Militärische Bereitschaft; Rüstungskontrolle und Nichtverbreitung von Atomwaffen; Militärische Unterstützung und Waffenlieferung an andere Staaten; Streitkräfte und deren Familien, Militärgericht; Kriegsheimkehrer und Veteranen; Militärische Beschaffung; Militärische Einrichtungen, Aufbau, Landerwerb; Bereitschaftsdienst und Reservisten; Militärbezogener Umweltschutz; Zivilverteidigung; Zivilbeschäftigte der Bundeswehr und der Verteidigungsindustrie, Schließung von militärischen Stützpunkten; Aufsicht über militärische Verträge und Lieferanten; Einsatz- und Kriegsangelegenheiten; Klagen gegen die Bundeswehr; Forschung und Entwicklung', 
'aeusseres':'Handelsverhandlungen, -streite, und -abkommen; Exportförderung und -regelung, Handelsbank; Internationale privatwirtschafliche Investitionen;  Produktivität und Wettbewerbsfähigkeit, Zahlungsbilanz; Einfuhrzoll und Einfuhrbeschränkungen; Wechselkurse; Internationale Umweltproblem und Ressourcenverwaltung; Internationale Finanz- und Wirtschaftsentwicklung; Sowietische Union, ehemalige Republiken und Osteuropa; Afrika; Westeuropa; Lateinamerika; Internationale Kanäle; Nordamerika; Asien, Pazifischer Raum, Australien und Japan; Naher Osten; Menschenrechte; Internationale Organisationen; Terrorismus und Entführung; Diplomaten, Botschaften, Bundesbürger im Ausland, ausländische Diplomaten in Deutschland, Pässe', 
'digitalisierung_technik':'DARA, ESA und Weltraumforschung; Satelliten und kommerzielle Nutzung des Weltraums; Technologietransfer, Internationale wissenschaftliche Kooperation; Telekommunikation und Infrastruktur; Wettervorhersage, Deutscher Wetterdienst, Meereskunde; Computerindustrie, Computersicherheit, Informationssicherheit; Forschung und Entwicklung', 
'soziales':'Familienangelegenheiten; Lebensmittelhilfe; Unterstützung für Haushalte mit geringem Einkommen; Altersfürsorge; Unterstützung für Behinderte und Arbeitsunfähige;  Soziale Dienste und Freiwilligenverbände; Kindergeld, Erziehungsgeld, und andere Unterstützung; städtisches Wohnungswesen; Stadtentwicklung; Ländliches Wohnungswesen; Ländliche Wirtschaftsentwicklung; Wohnungshilfe für Bürger niederen und mittleren Einkommens; Wohnungswesen für Senioren und Behinderten; Obdachlosigkeit',
'umwelt':'Trinkwassersicherheit; Abfallentsorgung; Verordnung, Behandlung, und Beseitigung von Sondermüll und Giftstoffen; Luftverschmutzung, Klimaerwärmung, und Lärmbelästigung; Wiederverwertung; Umweltrisiken und Schadstoffe in Gebäuden; Arten- und Waldschutz; Verschmutzung und Schutz von schiffbaren Wasserwegen und Küsten; Landschafts- und Wasserschutz; Forschung und Entwicklung; Parkanlagen, Denkmäler, historische Stätten, Erholungs- und; Naturschätze, Öffentliches Land, und Forstverwaltung; Forschung und Verwaltung von öffentlichen Gewässern', 
'europa':'Europäische Union und Gemeinsamer Markt', 
'arbeit':'Arbeitssicherheit und -schutz; Arbeitsfortbildung; Persönliche Leistungen für Arbeitnehmer; Gewerkschaften und Betriebsräte; Arbeitsbedingungen und Richtlinien; Kinder- und Jugendarbeit, Initiativen gegen Jugendarbeitslosigkeit; Mutterschutz, Erziehungsurlaub und Kinderbetreuung; Ausländische Arbeitnehmer', 
'kultur_medien_sport':'Kunst- und Kulturpflege; Sportregulierung und Wettaufsicht; Rundfunk, Fernsehen, und Kabelanbieter', 
'gesundheit':'Reform des Gesundheitswesens; Krankenversicherungsreform, -leistungen, und -kosten; Regulierung der Pharmaindustrie, Medizintechnik, und klinische Labore; Regulierung und Finanzierung medizinischer Einrichtungen; Finanzierung und Regulierung von Versicherungen und Ärzten; Medizinische Haftpflicht, Betrug und Missbrauch; Medizinische Arbeitskräfte und Ausbildung; Vorbeugung, übertragbare Krankheiten und Gesundheitsförderung; Säuglinge und Kinder; Geistige Gesundheit, Entwicklungsverzögerungen; Langzeitpflege, Heimversorgung, Rehabilitation, Pflege für unheilbare; Deckung und Kosten von verschreibungspflichtigen Medikamenten; Andere oder mehrere Leistungen und Maßnahmen; Tabakmissbrauch, Behandlung, und Aufklärung; Alkoholmissbrauch und Behandlung; Drogen- und Medikamentenmissbrauch, Aufklärung und Behandlung; Forschung und Entwicklung',
'landwirtschaft_ernaehrung':'Landwirtschaftlicher Handel; Staatszuschüsse für Bauern und Viehzüchter, landwirtschaftliche Katastrophenversicherung; Lebensmittelkontrolle und Sicherheit;  Landwirtschaftliche Vermarktung und Förderung; Tier- und Pflanzenerkrankung, und Schädlingsbekämpfung; Wohlergehen von Tieren, Umgang mit Tieren; Umweltprobleme durch die Landwirtschaf; Fischerei; Landwirtschaftliche Forschung und Entwicklung', 
'bildung_forschung':'Hochschulwesen; Grundschule und weiterführende Schule; Gleichstellung von Bildungschancen; Berufliche Ausbildung; Sonderschulen und Unterstützung bei Lernschwächen; Bildungs- und Exzellenzförderung; Forschung und Entwicklung', 
'wirtschaft_finanzen':'Inflation, Preise und Zinsen; Arbeitslosenquote, Arbeitslosigkeit; Zentralbank; Staatshaushalt und Staatsverschuldung; Steuern, Steuerpolitik und Steuerreformen;  Struktur- und Industriepolitik; Preiskontrolle und Stabilisation; Kernenergie und Atomkraftwerke; Elektrizität und Wasserkraft; Erdgas und Öl; Kohle; Alternative und erneuerbare Energie; Energieerhaltung und -Einsparung; Forschung und Entwicklung; Banken und Finanzaufsicht; Wertpapier- und Börsenaufsicht; Verbraucherfinanzen, Hypotheken, und Kreditkarten; Versicherungsaufsicht; Insolvenz; Fusions- und Kartellaufsicht, Unternehmensführung; Kleine und mittelständische Unternehmen; Urheberrechte und Patente; Tourismus; Verbraucherschutz und Verbraucherbetrug',
'none':'keine der Kategorien ist passend',
}

In [55]:
context_de = (
    "Du bist ein Politikwissenschaftler, welcher Dokumenten Politikfelder zuordnen soll. Die Dokumente fallen in die folgenden Kategorien. Eine Beschreibung des Inhalts der jeweiligen Kategorie erfolgt nach dem ':' Zeichen. \n"
)

context_de_labelonly = (
    "Du bist ein Politikwissenschaftler, welcher Dokumenten Politikfelder zuordnen soll. Die Dokumente fallen in die folgenden Kategorien. \n"
)

# instructions
labels_definitions_de = ""

for i in range(len(labels_de)):
    labels_definitions_de += f'{list(labels_de.values())[i]}:\n{list(definitions_de.values())[i]}\n'

# question
q_de_single = "In welches Politikfeld fällt dieses Dokument? Antworte nur mit einer einzelnen Kategorie."

q_de_multi= "In welche Politikfelder fällt dieses Dokument? Nenne bis zu fünf Kategorien."

In [58]:
context_en = (
    "You are a political scientist tasked with annotating documents into policy fields. The documents can be classified as one of the following categories. A description of each category is following the ':' sign. \n"
)

context_en_labelonly = (
    "You are a political scientist tasked with annotating documents into policy fields. The documents can be classified as one of the following categories.\n"
)

# instructions
labels_definitions_en = ""

for i in range(len(labels_en)):
    labels_definitions_en += f'{list(labels_en.values())[i]}:\n{list(definitions_en.values())[i]}\n'

# question
q_en_single = "Which policy field does this document belong to? Answer only with a single category."

q_en_multi= "Which policy field does this document belong to? You can name up to five categories."

In [47]:
def classify_text(text, system_message, model):

  # clean the text 
  text = re.sub(r'\s+', ' ', text).strip()

  # construct input

  messages = [
    # system prompt
    {"role": "system", "content": system_message},
    # user input
    {"role": "user", "content": text},
  ]

  response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0.0,
    seed=SEED
  )
  
  result = response.choices[0].message.content
  
  return result

In [48]:
MODEL = 'gpt-4o-2024-08-06' # currently the latest version of GPT-4o (on 2024-09-25)

In [49]:
# German prompt, single label
classifications_de_single = [classify_text(text, system_message = f"{context_de}\n{labels_definitions_de}\n\n{q_de_single}", model=MODEL) for text in test_input['_source.text']]
classifications_de_single

['Bürger- und Freiheitsrechte, Minderheitsrechte',
 'Arbeitwesen und Arbeitsmigration',
 'Verteidigungspolitik',
 'Transport und Verkehr',
 'keine Kategorie',
 'Innenpolitik und Sicherheit',
 'keine Kategorie',
 'Verteidigungspolitik',
 'Volkswirtschaft, Energie und Bankwesen',
 'Bürger- und Freiheitsrechte, Minderheitsrechte']

In [50]:
# German prompt, multi label
classifications_de_multi = [classify_text(text, system_message = f"{context_de}\n{labels_definitions_de}\n\n{q_de_multi}", model=MODEL) for text in test_input['_source.text']]
classifications_de_multi

['Bürger- und Freiheitsrechte, Minderheitsrechte; Sozialpolitik; Innenpolitik und Sicherheit',
 'Arbeitwesen und Arbeitsmigration, Bildung und Forschung, Bürger- und Freiheitsrechte, Minderheitsrechte',
 '1. Verteidigungspolitik\n2. Digitalisierung, Wissenschaft und Technologie\n3. Europapolitik',
 'Transport und Verkehr',
 'keine Kategorie',
 'Innenpolitik und Sicherheit',
 'keine Kategorie',
 '1. Verteidigungspolitik: Der Rücktransport deutscher Staatsbürger aus Wuhan könnte in den Zuständigkeitsbereich der Bundeswehr fallen, insbesondere wenn militärische Ressourcen oder Unterstützung genutzt werden.\n2. Gesundheitswesen: Da es sich um den Rücktransport im Zusammenhang mit dem Coronavirus handelt, ist auch das Gesundheitswesen relevant.\n3. Außenpolitik: Der Rücktransport von Staatsbürgern aus dem Ausland betrifft auch die Außenpolitik, insbesondere in Krisensituationen.\n4. Innenpolitik und Sicherheit: Die Koordination und Durchführung solcher Rücktransporte kann auch Aspekte der i

In [121]:
test_input['classification_de_single'] = classifications_de_single
test_input['classification_de_multi'] = classifications_de_multi
test_input

,_id,_source.text,label,classification_de_single,classification_de_multi
52,1038719233448923136,Nächstes Wochenende ist #Linke Woche der Zukun...,gesellschaft soziales,"Bürger- und Freiheitsrechte, Minderheitsrechte","Bürger- und Freiheitsrechte, Minderheitsrechte..."
54,1061529034750914562,Was erleben die Teilnehmenden im Programm Beru...,gesellschaft arbeit,Arbeitwesen und Arbeitsmigration,"Arbeitwesen und Arbeitsmigration, Bildung und ..."
89,1531158644460556291,Das #sondervermoegen wird nur für die Ausrüstu...,verteidigung digitalisierung_technik europa,Verteidigungspolitik,"1. Verteidigungspolitik\n2. Digitalisierung, W..."
2,1295351591890018306,Es geht um #KfZZulassung. https://t.co/DClf0npcfu,verkehr innere_sicherheit,Transport und Verkehr,Transport und Verkehr
47,981211216470265857,"100% Zustimmung! Jetzt ist die Chance da, unse...",gesellschaft bildung_forschung,keine Kategorie,keine Kategorie
127,1423649264480276480,Heute haben @FranziskaGiffey und @RaedSalehBer...,soziales arbeit wirtschaft_finanzen,Innenpolitik und Sicherheit,Innenpolitik und Sicherheit
40,1633726496534609921,Herzliche Einladung für Samstag nach #Heidenhe...,gesellschaft aeusseres wirtschaft_finanzen,keine Kategorie,keine Kategorie
88,1223602895377108992,#Statement II - Verteidigungsministerin @akk i...,verteidigung gesundheit,Verteidigungspolitik,1. Verteidigungspolitik: Der Rücktransport deu...
141,1014589654937587712,Die letzten beiden für heute...!\r\nÜbrigens: ...,umwelt wirtschaft_finanzen,"Volkswirtschaft, Energie und Bankwesen","1. Volkswirtschaft, Energie und Bankwesen: Alt..."
42,1052097903752302593,Heute ist der Tag der betrieblichen #Entgeltgl...,gesellschaft arbeit,"Bürger- und Freiheitsrechte, Minderheitsrechte",Arbeitwesen und Arbeitsmigration


The model does an okay job on classifying the inputs. However, it is struggling with implicit meaning and the multi-label output is somewhat inconsistent in its formatting

In [68]:
# Try dropping the definitions from the prompt in order to save on tokens in the input
classifications_de_single_labelonly = [classify_text(text, system_message = f"{context_de_labelonly}\n{'\n'.join(labels_de.values())}\n\n{q_de_single}", model=MODEL) for text in test_input['_source.text']]
classifications_de_single_labelonly

['Innenpolitik und Sicherheit',
 'Arbeitwesen und Arbeitsmigration',
 'Verteidigungspolitik',
 'Transport und Verkehr',
 'keine Kategorie',
 'Innenpolitik und Sicherheit',
 'keine Kategorie',
 'Verteidigungspolitik',
 'Umweltpolitik',
 'Sozialpolitik']

In [122]:
test_input['classifications_de_single_labelonly'] = classifications_de_single_labelonly
test_input

,_id,_source.text,label,classification_de_single,classification_de_multi,classifications_de_single_labelonly
52,1038719233448923136,Nächstes Wochenende ist #Linke Woche der Zukun...,gesellschaft soziales,"Bürger- und Freiheitsrechte, Minderheitsrechte","Bürger- und Freiheitsrechte, Minderheitsrechte...",Innenpolitik und Sicherheit
54,1061529034750914562,Was erleben die Teilnehmenden im Programm Beru...,gesellschaft arbeit,Arbeitwesen und Arbeitsmigration,"Arbeitwesen und Arbeitsmigration, Bildung und ...",Arbeitwesen und Arbeitsmigration
89,1531158644460556291,Das #sondervermoegen wird nur für die Ausrüstu...,verteidigung digitalisierung_technik europa,Verteidigungspolitik,"1. Verteidigungspolitik\n2. Digitalisierung, W...",Verteidigungspolitik
2,1295351591890018306,Es geht um #KfZZulassung. https://t.co/DClf0npcfu,verkehr innere_sicherheit,Transport und Verkehr,Transport und Verkehr,Transport und Verkehr
47,981211216470265857,"100% Zustimmung! Jetzt ist die Chance da, unse...",gesellschaft bildung_forschung,keine Kategorie,keine Kategorie,keine Kategorie
127,1423649264480276480,Heute haben @FranziskaGiffey und @RaedSalehBer...,soziales arbeit wirtschaft_finanzen,Innenpolitik und Sicherheit,Innenpolitik und Sicherheit,Innenpolitik und Sicherheit
40,1633726496534609921,Herzliche Einladung für Samstag nach #Heidenhe...,gesellschaft aeusseres wirtschaft_finanzen,keine Kategorie,keine Kategorie,keine Kategorie
88,1223602895377108992,#Statement II - Verteidigungsministerin @akk i...,verteidigung gesundheit,Verteidigungspolitik,1. Verteidigungspolitik: Der Rücktransport deu...,Verteidigungspolitik
141,1014589654937587712,Die letzten beiden für heute...!\r\nÜbrigens: ...,umwelt wirtschaft_finanzen,"Volkswirtschaft, Energie und Bankwesen","1. Volkswirtschaft, Energie und Bankwesen: Alt...",Umweltpolitik
42,1052097903752302593,Heute ist der Tag der betrieblichen #Entgeltgl...,gesellschaft arbeit,"Bürger- und Freiheitsrechte, Minderheitsrechte",Arbeitwesen und Arbeitsmigration,Sozialpolitik


In [70]:
classifications_de_multi_labelonly = [classify_text(text, system_message = f"{context_de_labelonly}\n{'\n'.join(labels_de.values())}\n\n{q_de_multi}", model=MODEL) for text in test_input['_source.text']]

,_id,_source.text,label,classification_de_single,classification_de_multi,classifications_de_single_labelonly,classifications_de_multi_labelonly
52,1038719233448923136,Nächstes Wochenende ist #Linke Woche der Zukun...,gesellschaft soziales,"Bürger- und Freiheitsrechte, Minderheitsrechte","Bürger- und Freiheitsrechte, Minderheitsrechte...",Innenpolitik und Sicherheit,Innenpolitik und Sicherheit \nSozialpolitik ...
54,1061529034750914562,Was erleben die Teilnehmenden im Programm Beru...,gesellschaft arbeit,Arbeitwesen und Arbeitsmigration,"Arbeitwesen und Arbeitsmigration, Bildung und ...",Arbeitwesen und Arbeitsmigration,Arbeitwesen und Arbeitsmigration \nBildung un...
89,1531158644460556291,Das #sondervermoegen wird nur für die Ausrüstu...,verteidigung digitalisierung_technik europa,Verteidigungspolitik,"1. Verteidigungspolitik\n2. Digitalisierung, W...",Verteidigungspolitik,Verteidigungspolitik \nInnenpolitik und Siche...
2,1295351591890018306,Es geht um #KfZZulassung. https://t.co/DClf0npcfu,verkehr innere_sicherheit,Transport und Verkehr,Transport und Verkehr,Transport und Verkehr,Transport und Verkehr
47,981211216470265857,"100% Zustimmung! Jetzt ist die Chance da, unse...",gesellschaft bildung_forschung,keine Kategorie,keine Kategorie,keine Kategorie,Innenpolitik und Sicherheit \nBürger- und Fre...
127,1423649264480276480,Heute haben @FranziskaGiffey und @RaedSalehBer...,soziales arbeit wirtschaft_finanzen,Innenpolitik und Sicherheit,Innenpolitik und Sicherheit,Innenpolitik und Sicherheit,Innenpolitik und Sicherheit
40,1633726496534609921,Herzliche Einladung für Samstag nach #Heidenhe...,gesellschaft aeusseres wirtschaft_finanzen,keine Kategorie,keine Kategorie,keine Kategorie,Innenpolitik und Sicherheit
88,1223602895377108992,#Statement II - Verteidigungsministerin @akk i...,verteidigung gesundheit,Verteidigungspolitik,1. Verteidigungspolitik: Der Rücktransport deu...,Verteidigungspolitik,"Gesundheitswesen, Verteidigungspolitik, Außenp..."
141,1014589654937587712,Die letzten beiden für heute...!\r\nÜbrigens: ...,umwelt wirtschaft_finanzen,"Volkswirtschaft, Energie und Bankwesen","1. Volkswirtschaft, Energie und Bankwesen: Alt...",Umweltpolitik,"Umweltpolitik \nVolkswirtschaft, Energie und ..."
42,1052097903752302593,Heute ist der Tag der betrieblichen #Entgeltgl...,gesellschaft arbeit,"Bürger- und Freiheitsrechte, Minderheitsrechte",Arbeitwesen und Arbeitsmigration,Sozialpolitik,"Sozialpolitik, Arbeitwesen und Arbeitsmigration"


In [123]:
test_input['classifications_de_multi_labelonly'] = classifications_de_multi_labelonly
test_input

,_id,_source.text,label,classification_de_single,classification_de_multi,classifications_de_single_labelonly,classifications_de_multi_labelonly
52,1038719233448923136,Nächstes Wochenende ist #Linke Woche der Zukun...,gesellschaft soziales,"Bürger- und Freiheitsrechte, Minderheitsrechte","Bürger- und Freiheitsrechte, Minderheitsrechte...",Innenpolitik und Sicherheit,Innenpolitik und Sicherheit \nSozialpolitik ...
54,1061529034750914562,Was erleben die Teilnehmenden im Programm Beru...,gesellschaft arbeit,Arbeitwesen und Arbeitsmigration,"Arbeitwesen und Arbeitsmigration, Bildung und ...",Arbeitwesen und Arbeitsmigration,Arbeitwesen und Arbeitsmigration \nBildung un...
89,1531158644460556291,Das #sondervermoegen wird nur für die Ausrüstu...,verteidigung digitalisierung_technik europa,Verteidigungspolitik,"1. Verteidigungspolitik\n2. Digitalisierung, W...",Verteidigungspolitik,Verteidigungspolitik \nInnenpolitik und Siche...
2,1295351591890018306,Es geht um #KfZZulassung. https://t.co/DClf0npcfu,verkehr innere_sicherheit,Transport und Verkehr,Transport und Verkehr,Transport und Verkehr,Transport und Verkehr
47,981211216470265857,"100% Zustimmung! Jetzt ist die Chance da, unse...",gesellschaft bildung_forschung,keine Kategorie,keine Kategorie,keine Kategorie,Innenpolitik und Sicherheit \nBürger- und Fre...
127,1423649264480276480,Heute haben @FranziskaGiffey und @RaedSalehBer...,soziales arbeit wirtschaft_finanzen,Innenpolitik und Sicherheit,Innenpolitik und Sicherheit,Innenpolitik und Sicherheit,Innenpolitik und Sicherheit
40,1633726496534609921,Herzliche Einladung für Samstag nach #Heidenhe...,gesellschaft aeusseres wirtschaft_finanzen,keine Kategorie,keine Kategorie,keine Kategorie,Innenpolitik und Sicherheit
88,1223602895377108992,#Statement II - Verteidigungsministerin @akk i...,verteidigung gesundheit,Verteidigungspolitik,1. Verteidigungspolitik: Der Rücktransport deu...,Verteidigungspolitik,"Gesundheitswesen, Verteidigungspolitik, Außenp..."
141,1014589654937587712,Die letzten beiden für heute...!\r\nÜbrigens: ...,umwelt wirtschaft_finanzen,"Volkswirtschaft, Energie und Bankwesen","1. Volkswirtschaft, Energie und Bankwesen: Alt...",Umweltpolitik,"Umweltpolitik \nVolkswirtschaft, Energie und ..."
42,1052097903752302593,Heute ist der Tag der betrieblichen #Entgeltgl...,gesellschaft arbeit,"Bürger- und Freiheitsrechte, Minderheitsrechte",Arbeitwesen und Arbeitsmigration,Sozialpolitik,"Sozialpolitik, Arbeitwesen und Arbeitsmigration"


The output does suffer a little bit, but multilabel classification seems to remedy this in part. Some of the classifications are hit-or-miss though, with multiple labels leading to both more true and false positives (esp. with no definitions)

In [82]:
# Reformulate as a single class problem (T/F), run over all classes
context_de_singleclass = (
    "Du bist ein Politikwissenschaftler, welcher Dokumenten Politikfelder zuordnen soll. Entscheide, ob das Dokument in diese Kategorie fällt.\n"
)

q_de_singleclass = "Fällt das Dokument in diese Kategorie? Antworte nur mit TRUE falls ja und nur mit FALSE falls nein."


results = {}
for label in list(labels_de.values())[0:15]: # drop "none" category
    results[label] = [classify_text(text, system_message = f"{context_de_singleclass}\n{label}\n{q_de_singleclass}", model=MODEL) for text in test_input['_source.text']]

    

In [130]:
results_df = pd.DataFrame(results)
results_df['_id'] = test_input.reset_index()['_id']
results_df = results_df.melt(id_vars='_id', var_name = "classification_de_singleclass")

results_df = results_df[results_df.value == "TRUE"] # only keep tweets labeled as a certain category (drop False values)

# concatenate the labels by _id
results_df = results_df.groupby('_id')['classification_de_singleclass'].apply(lambda x: "; ".join(x)).reset_index()

# join results_df on test_input on _id
comp_results = test_input.merge(results_df, on="_id")
comp_results

,_id,_source.text,label,classification_de_single,classification_de_multi,classifications_de_single_labelonly,classifications_de_multi_labelonly,classification_de_singleclass
0,1061529034750914562,Was erleben die Teilnehmenden im Programm Beru...,gesellschaft arbeit,Arbeitwesen und Arbeitsmigration,"Arbeitwesen und Arbeitsmigration, Bildung und ...",Arbeitwesen und Arbeitsmigration,Arbeitwesen und Arbeitsmigration \nBildung un...,Sozialpolitik; Arbeitwesen und Arbeitsmigration
1,1531158644460556291,Das #sondervermoegen wird nur für die Ausrüstu...,verteidigung digitalisierung_technik europa,Verteidigungspolitik,"1. Verteidigungspolitik\n2. Digitalisierung, W...",Verteidigungspolitik,Verteidigungspolitik \nInnenpolitik und Siche...,Innenpolitik und Sicherheit; Verteidigungspoli...
2,1295351591890018306,Es geht um #KfZZulassung. https://t.co/DClf0npcfu,verkehr innere_sicherheit,Transport und Verkehr,Transport und Verkehr,Transport und Verkehr,Transport und Verkehr,Transport und Verkehr
3,1223602895377108992,#Statement II - Verteidigungsministerin @akk i...,verteidigung gesundheit,Verteidigungspolitik,1. Verteidigungspolitik: Der Rücktransport deu...,Verteidigungspolitik,"Gesundheitswesen, Verteidigungspolitik, Außenp...",Gesundheitswesen
4,1014589654937587712,Die letzten beiden für heute...!\r\nÜbrigens: ...,umwelt wirtschaft_finanzen,"Volkswirtschaft, Energie und Bankwesen","1. Volkswirtschaft, Energie und Bankwesen: Alt...",Umweltpolitik,"Umweltpolitik \nVolkswirtschaft, Energie und ...",Umweltpolitik
5,1052097903752302593,Heute ist der Tag der betrieblichen #Entgeltgl...,gesellschaft arbeit,"Bürger- und Freiheitsrechte, Minderheitsrechte",Arbeitwesen und Arbeitsmigration,Sozialpolitik,"Sozialpolitik, Arbeitwesen und Arbeitsmigration",Sozialpolitik


Results are worse when looping through the categories, as not all documents get a category (implicitly landing 4/10 texts in the "none" category) => better to give all available categories to choose from